In [1]:
import pandas as pd

df = pd.read_parquet("/content/drive/MyDrive/Projet_Deep_learning/Api_yaho/eur_usd_max_data.parquet")


,Date,Open,High,Low,Close
0,2003-12-01,1.203398,1.204007,1.194401,1.196501
1,2003-12-02,1.196101,1.210903,1.194600,1.208897
2,2003-12-03,1.209000,1.213003,1.207700,1.212298
3,2003-12-04,1.212004,1.214403,1.204398,1.208094
4,2003-12-05,1.207802,1.219096,1.206593,1.218695


In [3]:
df.head(15)

,Date,Open,High,Low,Close
0,2003-12-01,1.203398,1.204007,1.194401,1.196501
1,2003-12-02,1.196101,1.210903,1.194600,1.208897
2,2003-12-03,1.209000,1.213003,1.207700,1.212298
3,2003-12-04,1.212004,1.214403,1.204398,1.208094
4,2003-12-05,1.207802,1.219096,1.206593,1.218695
5,2003-12-08,1.216797,1.224005,1.215407,1.222001
6,2003-12-09,1.222105,1.227702,1.219795,1.224995
7,2003-12-10,1.224905,1.226603,1.216205,1.219096
8,2003-12-11,1.219096,1.223496,1.212298,1.222404
9,2003-12-12,1.222703,1.230603,1.221300,1.227898


In [4]:
df['Date'] = pd.to_datetime(df['Date']).dt.date

In [5]:
import pandas as pd
import numpy as np

df['diff_open_close'] = df['Open'] - df['Close']

if 'Close' in df.columns:
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()

    df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()

    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()

def calculate_rsi(df, window=14): 
    delta = df['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

if 'Close' in df.columns:
    df['RSI'] = calculate_rsi(df)

if 'High' in df.columns and 'Low' in df.columns:
    df['price_range'] = df['High'] - df['Low']

def calculate_atr(df, window=14):  
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = ranges.max(axis=1)
    atr = true_range.rolling(window=window, min_periods=1).mean()
    return atr

if 'High' in df.columns and 'Low' in df.columns and 'Close' in df.columns:
    df['ATR'] = calculate_atr(df)

if 'High' in df.columns:
    df['SMA_High_10'] = df['High'].rolling(window=10).mean()

if 'Low' in df.columns:
    df['SMA_Low_10'] = df['Low'].rolling(window=10).mean()

def calculate_stochastic_oscillator(df, window=10):
    low_min = df['Low'].rolling(window=window).min()
    high_max = df['High'].rolling(window=window).max()
    stochastic = (df['Close'] - low_min) / (high_max - low_min) * 100
    return stochastic

if 'High' in df.columns and 'Low' in df.columns and 'Close' in df.columns:
    df['Stochastic_Oscillator'] = calculate_stochastic_oscillator(df)

df['signal'] = 'hold'

if 'MACD' in df.columns and 'Signal_Line' in df.columns:
    df.loc[(df['MACD'] > df['Signal_Line']), 'signal'] = 'buy'
    df.loc[(df['MACD'] < df['Signal_Line']), 'signal'] = 'sell'

if 'RSI' in df.columns:
    df.loc[(df['RSI'] < 30) & (df['signal'] == 'hold'), 'signal'] = 'buy'
    df.loc[(df['RSI'] > 70) & (df['signal'] == 'hold'), 'signal'] = 'sell'

if 'SMA_10' in df.columns and 'SMA_50' in df.columns:
    df.loc[(df['SMA_10'] > df['SMA_50']) & (df['signal'] == 'hold'), 'signal'] = 'buy'
    df.loc[(df['SMA_10'] < df['SMA_50']) & (df['signal'] == 'hold'), 'signal'] = 'sell'

if 'ATR' in df.columns:
    df.loc[(df['ATR'] > df['ATR'].rolling(window=10).mean()) & (df['signal'] == 'hold'), 'signal'] = 'buy'
    df.loc[(df['ATR'] < df['ATR'].rolling(window=10).mean()) & (df['signal'] == 'hold'), 'signal'] = 'sell'

if 'Stochastic_Oscillator' in df.columns:
    df.loc[(df['Stochastic_Oscillator'] < 20) & (df['signal'] == 'hold'), 'signal'] = 'buy'
    df.loc[(df['Stochastic_Oscillator'] > 80) & (df['signal'] == 'hold'), 'signal'] = 'sell'



In [6]:
df.shape

(5407, 19)

In [7]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'diff_open_close', 'SMA_10',
       'SMA_50', 'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'RSI',
       'price_range', 'ATR', 'SMA_High_10', 'SMA_Low_10',
       'Stochastic_Oscillator', 'signal'],
      dtype='object')

In [8]:
df = df.iloc[49:]
df.reset_index(drop=True, inplace=True)

In [9]:
df.shape

(5358, 19)

In [10]:
df.head()

,Date,Open,High,Low,Close,diff_open_close,SMA_10,SMA_50,EMA_12,EMA_26,MACD,Signal_Line,RSI,price_range,ATR,SMA_High_10,SMA_Low_10,Stochastic_Oscillator,signal
0,2004-02-06,1.257008,1.272605,1.251894,1.270002,-0.012995,1.252249,1.246749,1.255783,1.253821,0.001962,0.002065,62.869760,0.020711,0.015201,1.258932,1.245030,92.987439,sell
1,2004-02-09,1.265502,1.275705,1.265502,1.269406,-0.003904,1.254310,1.248207,1.257879,1.254976,0.002903,0.002233,54.708085,0.010203,0.014129,1.260392,1.246969,84.332695,buy
2,2004-02-10,1.269293,1.278707,1.267395,1.268504,0.000789,1.254791,1.249399,1.259513,1.255978,0.003536,0.002493,51.233947,0.011312,0.014194,1.261663,1.249240,76.387253,buy
3,2004-02-11,1.268295,1.284703,1.265807,1.283005,-0.014710,1.258672,1.250813,1.263128,1.257980,0.005148,0.003024,54.632473,0.018896,0.014801,1.263694,1.251620,96.549560,buy
4,2004-02-12,1.283302,1.284703,1.279607,1.281805,0.001497,1.262752,1.252287,1.266001,1.259745,0.006256,0.003671,61.714468,0.005096,0.013695,1.266895,1.255870,94.109880,buy


In [11]:

nombre_valeurs_nulles = df['SMA_50'].isnull().sum()

print(f"Le nombre : {nombre_valeurs_nulles}")


Le nombre : 0


In [12]:
def trans(dataset):
    for i in range(len(dataset)):
        if dataset[i] == "sell":
            dataset[i] = 0
        elif dataset[i] == "buy":
            dataset[i] = 1

trans(df['signal'])


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
<ipython-input-12-875465ce100e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[i] = 1
<ipython-input-12-875465ce100e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[i] = 1
<ipython-input-12-875465ce100e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[i] = 1
<ipython-input-12-875465ce100e>:6: SettingWithCopyWarning: 
A value is tr

In [13]:
df['Date'] = pd.to_datetime(df['Date'])

df = df.set_index('Date')


In [14]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'diff_open_close', 'SMA_10', 'SMA_50',
       'EMA_12', 'EMA_26', 'MACD', 'Signal_Line', 'RSI', 'price_range', 'ATR',
       'SMA_High_10', 'SMA_Low_10', 'Stochastic_Oscillator', 'signal'],
      dtype='object')

In [15]:
features = ["diff_open_close",
            "SMA_10", "SMA_50", "EMA_12", "EMA_26", "MACD", "Signal_Line",
            "RSI", "price_range", "ATR", "SMA_High_10", "Stochastic_Oscillator", "signal"]

df = df[features]

In [16]:
df.head()

,diff_open_close,SMA_10,SMA_50,EMA_12,EMA_26,MACD,Signal_Line,RSI,price_range,ATR,SMA_High_10,Stochastic_Oscillator,signal
Date,,,,,,,,,,,,,
2004-02-06,-0.012995,1.252249,1.246749,1.255783,1.253821,0.001962,0.002065,62.869760,0.020711,0.015201,1.258932,92.987439,0
2004-02-09,-0.003904,1.254310,1.248207,1.257879,1.254976,0.002903,0.002233,54.708085,0.010203,0.014129,1.260392,84.332695,1
2004-02-10,0.000789,1.254791,1.249399,1.259513,1.255978,0.003536,0.002493,51.233947,0.011312,0.014194,1.261663,76.387253,1
2004-02-11,-0.014710,1.258672,1.250813,1.263128,1.257980,0.005148,0.003024,54.632473,0.018896,0.014801,1.263694,96.549560,1
2004-02-12,0.001497,1.262752,1.252287,1.266001,1.259745,0.006256,0.003671,61.714468,0.005096,0.013695,1.266895,94.109880,1


In [17]:
import pandas as pd
import numpy as np

correlation_matrix = df.corr().abs()

print("Matrice de corrélation :")
print(correlation_matrix)

threshold = 0.9

high_correlation_pairs = np.where(correlation_matrix > threshold)
high_correlation_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y])
                          for x, y in zip(*high_correlation_pairs) if x != y and x < y]

print("\nPaires de variables fortement corrélées :")
for pair in high_correlation_pairs:
    print(pair)

variables_to_remove = set()
for var1, var2 in high_correlation_pairs:
    if var1 not in variables_to_remove and var2 not in variables_to_remove:
        variables_to_remove.add(var2)

reduced_df = df.drop(columns=list(variables_to_remove))

print("\nVariables supprimées :", variables_to_remove)
print("\nNouveau DataFrame sans les variables redondantes :")
print(reduced_df)


Paires de variables fortement corrélées :
('SMA_10', 'SMA_50')
('SMA_10', 'EMA_12')
('SMA_10', 'EMA_26')
('SMA_10', 'SMA_High_10')
('SMA_50', 'EMA_12')
('SMA_50', 'EMA_26')
('SMA_50', 'SMA_High_10')
('EMA_12', 'EMA_26')
('EMA_12', 'SMA_High_10')
('EMA_26', 'SMA_High_10')
('MACD', 'Signal_Line')

Variables supprimées : {'EMA_12', 'EMA_26', 'Signal_Line', 'SMA_50', 'SMA_High_10'}

Nouveau DataFrame sans les variables redondantes :


In [19]:
reduced_df.shape

(5358, 8)

In [20]:
reduced_df.columns

Index(['diff_open_close', 'SMA_10', 'MACD', 'RSI', 'price_range', 'ATR',
       'Stochastic_Oscillator', 'signal'],
      dtype='object')

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib  

features = ["diff_open_close", "SMA_10", "MACD", "RSI", "price_range", "ATR", "Stochastic_Oscillator"]
X = reduced_df[features].values
y = reduced_df['signal'].values

y = y.astype(int) 

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

model_RF = RandomForestClassifier(n_estimators=100, random_state=42)

model_RF.fit(X_train, y_train)

y_pred = model_RF.predict(X_test)

print(classification_report(y_test, y_pred, target_names=["Sell", "Buy"]))


              precision    recall  f1-score   support

        Sell       0.85      0.89      0.87       253
         Buy       0.90      0.86      0.88       283

    accuracy                           0.87       536
   macro avg       0.87      0.87      0.87       536
weighted avg       0.87      0.87      0.87       536



In [ ]:
import pickle 

with open('/content/drive/MyDrive/Projet_Deep_learning/models/model_RF.pkl', 'wb') as model_file:
    pickle.dump(model_RF, model_file)

with open('/content/drive/MyDrive/Projet_Deep_learning/models/scaler_RF.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)
